# Test ETradeBot

In [1]:
import ast
import sys
import keyring

In [2]:
sys.path.insert(0, '../')
from strategies.strategy import Strategy
from etrade.etrade import ETrade
from execute.execute import Execute
from bot import Bot

## Initialize

Log in to E-Trade, authenticate account, and initialize API objects.

In [3]:
# Retrieve credentials from keyring
consumer_key = keyring.get_password("etrade", "consumer_key")
consumer_secret = keyring.get_password("etrade", "consumer_secret")
sandbox_key = keyring.get_password("etrade", "sandbox_key")
sandbox_secret = keyring.get_password("etrade", "sandbox_secret")
web_username = keyring.get_password("etrade", "web_username")
web_password = keyring.get_password("etrade", "web_password")
account_id = keyring.get_password("etrade", "account_id")
etrade_cookie = ast.literal_eval(keyring.get_password("etrade", "cookie"))
account_id_key = keyring.get_password("etrade", "account_id_key")

In [4]:
# Define the strategy name
strategy_name = 'example_strategy'

In [5]:
# Configuration
dev=False  # Set to False for production, True for sandbox
headless=True  # Set to False to show browser window, True to hide
browser='chrome'  # Set to 'chrome' or 'edge'
preview=True  # Set to False to execute trades, True to preview
prints=True  # Set to True to print information to console

In [6]:
# Initialize API objects
etrade = ETrade(
    consumer_key, consumer_secret, web_username, web_password, account_id, etrade_cookie,
    sandbox_key, sandbox_secret, dev, headless, browser
)
strategy = Strategy(prints, strategy_name)
execute = Execute(etrade)

## Strategy

Run the custom strategy.

In [7]:
new_portfolio = strategy.run_strategy()

------------------------------------
Risk Free Rate: 3.9%

Maximum Sharpe Portfolio:
Sharpe Ratio: 0.2018

Weights:
SPY    30.00%
LQD    27.20%
TLT    24.50%
IWM    10.99%
Name: Maximum Sharpe Portfolio, dtype: object

Results:
Expected_Return         5.89%
Expected Volatility     9.84%
Sharpe_Ratio           0.2018
Name: Results, dtype: object

Long/Short Ratio:
Portfolio weights sum: 92.69%
Long weights sum: 92.69%
Short weights sum: 0.0%


In [8]:
new_portfolio

SPY    0.3000
LQD    0.2720
TLT    0.2450
IWM    0.1099
Name: Maximum Sharpe Portfolio, dtype: float64

## Current Portfolio

Get the current portfolio and buying power of the account from E-Tade.

In [9]:
# Get current portfolio
current_portfolio = etrade.get_portfolio_data(account_id_key)[[
    'pctOfPortfolio', 'quantity', 'positionType'
]]
current_portfolio

,pctOfPortfolio,quantity,positionType
symbolDescription,,,
PSCD,21.0855,23,LONG
IWF,43.1087,16,LONG
VONG,34.6680,50,LONG
VIOV,26.2465,33,LONG
IVE,6.2306,4,LONG
TMFC,-30.1445,-76,SHORT


In [10]:
# Get buying power
buying_power = etrade.get_buying_power(account_id_key, prints=prints)


Buying Power: $11,143.17


## Implement Strategy

Generate trades which will implement the custom strategy given the current portfolio. 

In [11]:
# Calculate new portfolio shares
new_portfolio_shares = execute.calculate_shares(new_portfolio, buying_power, prints=prints)


Trade Instructions:
SPY    $3,312.86
LQD    $2,988.49
TLT    $2,662.12
IWM    $1,195.50
Name: Trade Amounts, dtype: object

Position Value: $10,158.97
Leftover: $984.20

SPY     7
LQD    27
TLT    27
IWM     6
Name: Shares, dtype: object


In [12]:
# Preview or execute trades
print('')
if preview:
    print('Previewing trades...')
else:
    print('Executing trades...')
place_order = execute.execute_trades(
    current_portfolio, new_portfolio_shares, account_id_key, preview, prints
)


Previewing trades...
SELL 23 shares of PSCD
SELL 16 shares of IWF
SELL 50 shares of VONG
SELL 33 shares of VIOV
SELL 4 shares of IVE
BUY_TO_COVER -76 shares of TMFC
BUY 7 shares of SPY
BUY 27 shares of LQD
BUY 27 shares of TLT
BUY 6 shares of IWM


In [13]:
place_order

,orderAction,priceType,quantity,orderTerm,marketSession
symbol,,,,,
PSCD,SELL,MARKET,23,GOOD_FOR_DAY,REGULAR
IWF,SELL,MARKET,16,GOOD_FOR_DAY,REGULAR
VONG,SELL,MARKET,50,GOOD_FOR_DAY,REGULAR
VIOV,SELL,MARKET,33,GOOD_FOR_DAY,REGULAR
IVE,SELL,MARKET,4,GOOD_FOR_DAY,REGULAR
TMFC,BUY_TO_COVER,MARKET,76,GOOD_FOR_DAY,REGULAR
SPY,BUY,MARKET,7,GOOD_FOR_DAY,REGULAR
LQD,BUY,MARKET,27,GOOD_FOR_DAY,REGULAR
TLT,BUY,MARKET,27,GOOD_FOR_DAY,REGULAR
